# <font color='red'> Tic Tac Toe Complete Game Tree

In this notebook, I aim to implement the building of the complete game tree of tic-tac-toe. This game tree starts with an empty board and player X is the first player to move. The final goal is to calculate the number of nodes in this game tree, which indicates the total number of possible allowed states can be observed in tic-tac-toe.

In the following, 1 represents X player, -1 represents O, and 0 is used to represent a draw.

In [2]:
import numpy as np

First of all, a class to represent nodes. This class will be used later to build a tree, as a tree is a collection of nodes.
There are some important properties for a node class:
<ol>
1- Node number, which indicates the chronological order in which nodes have been created.<br>
2- The board state stored in the node.<br>
3- Parent of a node, which is the node from which the current board state is generated.<br>
4- Children of the node, which are states generated from the current node.<br>
5- An attribute to indicate whether the node is a leaf.<br>
6- Three attributes to show whether the current node indicates a state in which X is the winner, O is the winner or a draw occurs. Only one of them can be equal to one, the others must be zero. If the state is not a final state, i.e. the node is not a leaf, all of them would be zero.<br>
</ol>

In [3]:
class Node(object):
    
    def __init__(self, nodeNumber, state, parent = None):
        
        self.nodeNumber = nodeNumber
        self.state = state
        self.parent = parent
        self.children = []
        self.x = 0
        self.o = 0
        self.draw = 0
        
        if get_winner(self.get_state()) == 1 or get_winner(self.get_state()) == -1 or not (0 in self.get_state()):
            self.leaf = True
        else:
            self.leaf = False
            
        if(self.leaf == True):
            if get_winner(self.get_state()) == 1:
                self.x = 1
            
            if get_winner(self.get_state()) == -1:
                self.o = 1
            
            if get_winner(self.get_state()) == 0:
                self.draw = 1
                    
        
    def get_nodeNumber(self):
        return self.nodeNumber
    
    def get_state(self):
        return self.state
    
    def get_parent(self):
        return self.parent
    
    def add_children(self, newChild):
        self.children.append(newChild)
        
    def get_children(self):
        return self.children
            
            
    def get_leaf(self):
        return self.leaf
            
               
    def get_x(self):
        return self.x
            
                
    def get_o(self):
        return self.o
                             
                
    def get_draw(self):
        return self.draw

Secondly, a representative class for trees. This class gets its root as an argument. The objects used in this class are as follows:
<ol>
1- The root, which is an empty board, in case of the entire tree.<br>
2- nodes, a list of all nodes (as instances of the node class) in the tree.<br>
3- Counters for all winning board states for both players 'X' and 'O', and also for all board states leading to a draw. If the new node added to the tree is a final state, one of these counters will be increased by one, according to the final result.<br>
4- Number of all leaves in the tree.<br>
5- The branching factor of the tree, which is the average number of children in the tree.<br>
</ol>

There are also some class methods, e.g. a method to calculate branching factor and another one to add a new node to the tree.

In [4]:
class Tree(object):
    
    def __init__(self, root):
        
        self.root  = root
        self.nodes = [root]
        self.OWinsCounter = 0
        self.XWinsCounter = 0
        self.drawsCounter = 0
        self.leaves = 0
        self.branchingFactor = 0
    
    # calculating the branching factor    
    def calculate_branching_factor(self):
        for node in self.nodes:
            self.branchingFactor += len(node.get_children())
        self.branchingFactor /= self.get_size()
        return self.branchingFactor
    
    # adding a new node to the tree
    def add_node(self, node):
        # first, add the node to the tree
        self.nodes.append(node)
        # if the new node is a leaf, increase the counter for the winner by one (in case of a draw, the counter for draws), 
        # otherwise leave it unchanged
        self.OWinsCounter += node.get_o()
        self.XWinsCounter += node.get_x()
        self.drawsCounter += node.get_draw()
        # increasing the leaves counter by one, if the node is a leaf
        if(node.get_leaf()):
            self.leaves += 1
    
    # a method which gets the node number and returns the node
    def get_node(self,nodeNumber):
        return self.nodes[nodeNumber]
    
    # getting all nodes
    def get_all_nodes(self):
        return self.nodes
        
    # getting the size of the tree, to find the number of nodes
    def get_size(self):
        return len(self.nodes)
    
    # getting the number of wins by O
    def get_OWins(self):
        return self.OWinsCounter
    
    # getting the number of wins by X
    def get_XWins(self):
        return self.XWinsCounter
    
    # getting the number of draws
    def get_draws(self):
        return self.drawsCounter
    
    # getting the number of leaves
    def get_leaves(self):
        return self.leaves

In order to do further computations, some functions are defined out of the classes.

A function to check if the current move leads to one of the players wins the game:

In [5]:
def move_was_winning_move(S, p):
    
    # Check the horizontal lines to find a winner. Note that the sign of p (player) is the same with the sign of sum function
    if np.max((np.sum(S, axis=0)) * p) == 3:
        return True
    # Check the vertical lines to find a winner
    if np.max((np.sum(S, axis=1)) * p) == 3:
        return True
    # Check the diagonal lines to find a winner
    if (np.sum(np.diag(S)) * p) == 3:
        return True
    if (np.sum(np.diag(np.rot90(S))) * p) == 3:
        return True
    return False

For a leaf, the following function returns the winner: (Notes are similar to the function above)

In [6]:
def get_winner(S):
    
    if np.max((np.sum(S, axis=0))) == 3:
        return 1
    if np.min((np.sum(S, axis=0))) == -3:
        return -1
    if np.max((np.sum(S, axis=1))) == 3:
        return 1
    if np.min((np.sum(S, axis=1))) == -3:
        return -1
    if (np.sum(np.diag(S))) == 3:
        return 1
    if (np.sum(np.diag(S))) == -3:
        return -1
    if (np.sum(np.diag(np.rot90(S)))) == 3: 
        return 1
    if (np.sum(np.diag(np.rot90(S)))) == -3:
        return -1
    return 0

A function to build the complete tree; The arguments are as follows:
<ol>
1- The current player.<br>
2- The current game board.<br>
3- The current shape of the tree.<br>
4- The current node.<br>
</ol>

The difference between the current game board and the current node is that the current game board is the state of the board and has nothing to do with the tree, while the current node is a node in the tree (an instance of the node class), which stores the current game board as its state.

In [7]:
def buildTree(p, S, tree, parentNode):

    # a list to keep all successors of the current node
    succNodes = []
    # if S is not terminal: switch player & compute successors
    if not move_was_winning_move(S, p):
        # finding all empty cells in the board, where the current player can place his new move 
        rs, cs = np.where(S==0)
        # iterating over all empty cells
        for j in range(rs.size):
            # making a copy of the current state to be modified for making a successor
            Ssucc = np.copy(S)
            # at each iteration, one of the empty cells filled by the current player to produce a new board state for successor
            Ssucc[rs[j],cs[j]] = p
            # the number of the successor node, which contains the new board state, is equal to the size of the tree so far
            newnode = tree.get_size()
            # adding the number of the successor to the successors list
            succNodes.append(newnode)
            # creating the successor node
            childnode = Node(nodeNumber=newnode, state=Ssucc, parent=parentNode)
            # adding the successor node to the parent node
            tree.get_node(parentNode.get_nodeNumber()).add_children(childnode.get_nodeNumber())
            # adding the new node to the tree
            tree.add_node(childnode)
                
    # continue recursively for each suucessor
    p *= -1
    for n in succNodes:
        if tree.get_node(n).get_leaf() == False:
            buildTree(p, tree.get_node(n).get_state(), tree, tree.get_node(n))

Finally, in the main function the final results are shown:

In [8]:
if __name__ == "__main__":
    # First board state, with all cells empty
    S = np.zeros((3,3), dtype=int)
    # player X is the one to start
    p = 1
    # creating the first node to keep the first board state
    node = Node(nodeNumber=0, state=S)
    # initialization of the tree
    tree = Tree(node)
    # building the entire tree recursively
    buildTree(p, S, tree, tree.get_node(0))
     
    print("Number of nodes:")
    print(tree.get_size())
    print()
    print("\nNumber of states where X wins:")
    print(tree.get_XWins())
    print()
    print("\nNumber of states where O wins:")
    print(tree.get_OWins())
    print()
    print("\nNumber of draws:")
    print(tree.get_draws())
    print()
    print("\nNumber of leaves:")
    print(tree.get_leaves())
    print()
    print("\nBranching factor:")
    print(tree.calculate_branching_factor())
    print()

Number of nodes:
549946


Number of states where X wins:
131184


Number of states where O wins:
77904


Number of draws:
46080


Number of leaves:
255168


Branching factor:
0.9999981816396519

